In [1]:
import numpy as np
import random
import math

from matplotlib import pyplot as plt

In [11]:
# Paramètres
n = m = 18 # nombre de lignes = nombre de colonnes
initial_size = 2 # taille de la fourmilière initiale
max_food_quantity = 10 #quantité maximale de nourriture par source de nourriture
neigh = 1 #voisinage / champ de vision de chaque fourmi
pers_pher = 40 #persistance des phéromones dans l'environnement
life_time = 100 #durée de vie initiale des fourmis

In [12]:
def generate_simple_map(n, m): # n : lignes m : colonnes
    """Préconditions : m > 0 et n > 0, initial_size > 0
    Renvoie un monde de taille n*m avec une fourmilière au centre et un point de nourriture en périphérie"""
    map = np.array([[" " for i in range(0, n)] for j in range(0, n)]) # on crée la matrice vide (avec des " " partout)
    # on crée la fourmilière au centre
    for i in range(0, initial_size):
        for k in range(0, initial_size):
            map = np.delete(map, m*(n//2 - initial_size//2 + i) + m//2 - initial_size//2 + k) #on copie le monde en enlevant une à une les cases du carré de taille initial_size situé au centre
            map = np.insert(map, m*(n//2 - initial_size//2 + i) + m//2 - initial_size//2 + k, "h") #on remplace les cases initialement vide (avec un " ") par une case "fourmilière" (avec un "h" pour "home")
            map = np.array([[map[m*j + i] for i in range(0, m)] for j in range(0, n) ])
    
    # on crée un point de nourriture
    food_line = random.randint(0, 2*n//5 - 1) # on définit ainsi la périphérie comme étant les cases du bord de la matrice avec un largeur de n/5
    food_column = random.randint(0, 2*m//5 - 1)
    if food_line > n//5 - 1: # on est sur la périphérie du bas
        food_line = n - 2 - n//5 + food_line
    if food_column > m//5 - 1: # on est sur la périphérie de droite
        food_column = m - 2 - m//5 + food_column
    where = random.sample([food_line, food_column], 1) # on définit une périphérie où se mettre (horizontale ou verticale)
    if where == food_line: # on est sur une périphérie en haut ou en bas
        food_column = random.randint(0, m - 1) # on définit une case au hasard sur cette périphérie
    else : #on est sur une périphérie latérale
        food_line = random.randint(0, n - 1) # on définit une case au hasard sur cette périphérie
    # on place dans la matrice de point de nourriture de quantité aléatoire food_quantity et de position périphérique aléatoire (food_line, food_column)
    food_quantity = random.randint(1, max_food_quantity) # on attribue à ce point de nourriture une certaine quantité de nourriture
    print(food_quantity)
    map = np.delete(map, m*food_line + food_column)
    map = np.insert(map, m*food_line + food_column, food_quantity)
    map = np.array([[map[m*j + i] for i in range(0, m)] for j in range(0, n) ])       
    
    return map
print (generate_simple_map(n, m))

2
[[' ' '2' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'h' 'h' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'h' 'h' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' '

In [56]:
map1 = generate_simple_map(n, m)
map_pher_home1 = np.array([[0 for k in range(0, m)] for l in range(0, n)])
map_pher_food1 = np.array([[0 for k in range(0, m)] for l in range(0, n)])

print(map1)
print(map_pher_home1)
print(map_pher_food1)


def spatial_ants_neighborhood (map, a, b, neigh): #world : List[List[int]], a : int(ligne), b : int(colonne), neigh : int
    
    """Préconditions :0 <= a <= n-1  and 0 <= b <= m-1 and n >= 2*neigh and m >= 2*neigh
    Retourne la matrice des types des voisins de de l'individu en [a, b] avec a la ligne et b la colonne"""
    
    voisin = np.array([[" " for i in range(0, 2*neigh + 1)] for j in range(0, 2*neigh + 1)]) #grille vide de la taille du voisinage
    
    world_without_ind = np.delete(map, m*a + b) #on copie le monde sans l'individu
    world_without_ind = np.insert(world_without_ind, m*a + b, " ") #on remplace la place de l'individu par une case vide (avec un " ")
    world_without_ind = np.array([[world_without_ind[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
    

    if a - neigh >= 0 and a + neigh <= n - 1: # on teste les lignes, il n'y a pas de problème avec les bords (n-1 car les indices commencent à 0)
        if b - neigh >= 0 and b + neigh <= m - 1: #on teste les colonnes, il n'y a pas de problèmes avec les bords
            voisin = np.array([[world_without_ind[a-neigh + j, b-neigh + i] for i in range(0, 2*neigh + 1)] for j in range(0, 2*neigh + 1)]) 
        elif b - neigh < 0: # on atteint le bord gauche de la matrice
            voisin = np.array([[world_without_ind[a - neigh + j, i] for i in range(0, b + neigh + 1)] for j in range(0, 2*neigh + 1)])
        elif b + neigh > m - 1: # on atteint le bord droit
            voisin = np.array([[world_without_ind[a - neigh + j, i] for i in range(b - neigh, m)] for j in range(0, 2*neigh + 1)])
    elif a - neigh < 0 : # on atteint le bord haut
        if b - neigh >= 0 and b + neigh <= m - 1: #on teste les colonnes, il n'y a pas de problèmes avec les bords
            voisin = np.array([[world_without_ind[j, b-neigh + i] for i in range(0, 2*neigh + 1)] for j in range(0, a + neigh + 1)])
        elif b - neigh < 0: # on atteint le bord gauche de la matrice
            voisin = np.array([[world_without_ind[j, i] for i in range(0, b + neigh + 1)] for j in range(0, a + neigh + 1)])
        elif b + neigh > m - 1: # on atteint le bord droit
            voisin = np.array([[world_without_ind[j, i] for i in range(b - neigh, m)] for j in range(0, a + neigh + 1)])
    else : # a + neigh > n - 1
        if b - neigh >= 0 and b + neigh <= m - 1: #on teste les colonnes, il n'y a pas de problèmes avec les bords
            voisin = np.array([[world_without_ind[j, b-neigh + i] for i in range(0, 2*neigh + 1)] for j in range(a - neigh, n)]) #on copie 
        elif b - neigh < 0: # on atteint le bord gauche de la matrice
            voisin = np.array([[world_without_ind[j, i] for i in range(0, b + neigh + 1)] for j in range(a - neigh, n)])
        elif b + neigh > m - 1: # on atteint le bord droit
            voisin = np.array([[world_without_ind[j, i] for i in range(b - neigh, m)] for j in range(a - neigh, n)])
    return voisin

def spatial_pher_neighborhood (map, a, b, neigh): #world : List[List[int]], a : int(ligne), b : int(colonne), neigh : int
    
    """Préconditions :0 <= a <= n-1  and 0 <= b <= m-1 and n >= 2*neigh and m >= 2*neigh
    Retourne la matrice des types des voisins de de l'individu en [a, b] avec a la ligne et b la colonne"""
    
    voisin = np.array([[0 for i in range(0, 2*neigh + 1)] for j in range(0, 2*neigh + 1)]) #grille vide de la taille du voisinage
    
    world_without_ind = np.delete(map, m*a + b) #on copie le monde sans l'individu
    world_without_ind = np.insert(world_without_ind, m*a + b, 0) #on remplace la place de l'individu par une case vide (avec un " ")
    world_without_ind = np.array([[world_without_ind[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
    

    if a - neigh >= 0 and a + neigh <= n - 1: # on teste les lignes, il n'y a pas de problème avec les bords (n-1 car les indices commencent à 0)
        if b - neigh >= 0 and b + neigh <= m - 1: #on teste les colonnes, il n'y a pas de problèmes avec les bords
            voisin = np.array([[world_without_ind[a-neigh + j, b-neigh + i] for i in range(0, 2*neigh + 1)] for j in range(0, 2*neigh + 1)]) 
        elif b - neigh < 0: # on atteint le bord gauche de la matrice
            voisin = np.array([[world_without_ind[a - neigh + j, i] for i in range(0, b + neigh + 1)] for j in range(0, 2*neigh + 1)])
        elif b + neigh > m - 1: # on atteint le bord droit
            voisin = np.array([[world_without_ind[a - neigh + j, i] for i in range(b - neigh, m)] for j in range(0, 2*neigh + 1)])
    elif a - neigh < 0 : # on atteint le bord haut
        if b - neigh >= 0 and b + neigh <= m - 1: #on teste les colonnes, il n'y a pas de problèmes avec les bords
            voisin = np.array([[world_without_ind[j, b-neigh + i] for i in range(0, 2*neigh + 1)] for j in range(0, a + neigh + 1)])
        elif b - neigh < 0: # on atteint le bord gauche de la matrice
            voisin = np.array([[world_without_ind[j, i] for i in range(0, b + neigh + 1)] for j in range(0, a + neigh + 1)])
        elif b + neigh > m - 1: # on atteint le bord droit
            voisin = np.array([[world_without_ind[j, i] for i in range(b - neigh, m)] for j in range(0, a + neigh + 1)])
    else : # a + neigh > n - 1
        if b - neigh >= 0 and b + neigh <= m - 1: #on teste les colonnes, il n'y a pas de problèmes avec les bords
            voisin = np.array([[world_without_ind[j, b-neigh + i] for i in range(0, 2*neigh + 1)] for j in range(a - neigh, n)]) #on copie 
        elif b - neigh < 0: # on atteint le bord gauche de la matrice
            voisin = np.array([[world_without_ind[j, i] for i in range(0, b + neigh + 1)] for j in range(a - neigh, n)])
        elif b + neigh > m - 1: # on atteint le bord droit
            voisin = np.array([[world_without_ind[j, i] for i in range(b - neigh, m)] for j in range(a - neigh, n)])
    return voisin

def ants (map, nb_ants):
    """Precondition : nb_ants >= 0
    Fait apparaitre nb_ants fourmis autour de la fourmilière"""
    ants_map = map
    for k in range(0, n):
        for l in range(0, m):
            if ants_map[k, l] == " ": #on vérifie que la case est vide
                for o in spatial_ants_neighborhood(map, k, l, neigh):
                    for p in o:
                        if p == "h" and nb_ants > 0 : #on regarde si il y a une case fourmilière dans le voisinage
                            if ants_map[k, l] == " ": #on vérifie que la case est vide
                                ants_map = np.delete(ants_map, m*k + l) #on copie le monde en enlevant la case ayant la fourmilière pour voisin
                                ants_map = np.insert(ants_map, m*k + l, "f") #on remplace les cases initialement vide (avec un 0) par une case "fourmi" (avec un 2)
                                ants_map = np.array([[ants_map[m*j + i] for i in range(0, m)] for j in range(0, n) ])
                                nb_ants = nb_ants - 1
    q = 0                            
    while nb_ants > 0: #la fourmilière est entourée de fourmis
        q = q + 1
        for k in range(0, n):
            for l in range(0, m):
                if ants_map[k, l] == " ": #on vérifie que la case est vide
                    for o in spatial_ants_neighborhood(map, k, l, neigh + q):
                        for p in o:
                            if p == "h" and nb_ants > 0 : #on regarde si il y a une case fourmilière dans le voisinage + q
                                if ants_map[k, l] == " ": #on vérifie que la case est vide
                                    ants_map = np.delete(ants_map, m*k + l) #on copie le monde en enlevant la case ayant la fourmilière pour voisin
                                    ants_map = np.insert(ants_map, m*k + l, "f") #on remplace les cases initialement vide (avec un " ") par une case "fourmi" (avec un f)
                                    ants_map = np.array([[ants_map[m*j + i] for i in range(0, m)] for j in range(0, n) ])
                                    nb_ants = nb_ants - 1                        
    return ants_map


def clock_map(map):
    """Renvoie la carte des durées de vie des fourmis"""
    clock = list()
    for i in range(0, n):
        for j in range(0, m):
            if map[i, j] == "f" or map[i, j] == "s":
                clock.append(life_time)
            else :
                clock.append(0)
    clock = np.array(clock)
    return clock

print(clock_map(ants(generate_simple_map(n, m), 10)))

def no_pheromone(map, a, b):
    """Précondition : 0 >= a >= n, 0 >= b >= m
    Renvoie True si il n'y a pas des phéromones dans le voisinage de la case et False sinon."""
    for i in spatial_pher_neighborhood(map, a, b, 1):
        for j in i:
            if j != 0 :
                return False
    return True

def max_pheromone(map_pher, a, b):
    """Renvoie la case du voisinage de [a, b] ayant la plus grande valeur de phéromone"""
    maxi = 0
    maxi_line = 0
    maxi_column = 0
    if 0 < a < n-1 and 0 < b < m-1 : #pas de problème avec les bords
        for i in range(0, 3):
            for j in range(0, 3):
                if spatial_pher_neighborhood(map_pher, a, b, 1)[i, j] > maxi:
                    maxi == spatial_pher_neighborhood(map_pher, a, b, 1)[i, j]
                    maxi_line == i
                    maxi_column == j
    if a == 0 and 0 < b < m-1: # on a atteint le bord haut
        for i in range(0, 2):
            for j in range(0, 3):
                if spatial_pher_neighborhood(map_pher, a, b, 1)[i, j] > maxi:
                    maxi == spatial_pher_neighborhood(map_pher, a, b, 1)[i, j]
                    maxi_line == i
                    maxi_column == j
    if a == 0 and b == 0: # on a atteint le bord haut gauche
        for i in range(0, 2):
            for j in range(0, 2):
                if spatial_pher_neighborhood(map_pher, a, b, 1)[i, j] > maxi:
                    maxi == spatial_pher_neighborhood(map_pher, a, b, 1)[i, j]
                    maxi_line == i
                    maxi_column == j      
    if a == 0 and b == m-1: #on a atteint le bord haut droit
        for i in range(0, 2):
            for j in range(0, 2):
                if spatial_pher_neighborhood(map_pher, a, b, 1)[i, j] > maxi:
                    maxi == spatial_pher_neighborhood(map_pher, a, b, 1)[i, j]
                    maxi_line == i
                    maxi_column == j
    if a == n-1 and 0 < b < m-1: # on a atteint le bord bas
        for i in range(0, 2):
            for j in range(0, 3):
                if spatial_pher_neighborhood(map_pher, a, b, 1)[i, j] > maxi:
                    maxi == spatial_pher_neighborhood(map_pher, a, b, 1)[i, j]
                    maxi_line == i
                    maxi_column == j
    if a == n-1 and b == 0: # on a atteint le bord bas gauche
        for i in range(0, 2):
            for j in range(0, 2):
                if spatial_pher_neighborhood(map_pher, a, b, 1)[i, j] > maxi:
                    maxi == spatial_pher_neighborhood(map_pher, a, b, 1)[i, j]
                    maxi_line == i
                    maxi_column == j
    if a == n-1 and b == m-1: #on a atteint le bord bas droit
        for i in range(0, 2):
            for j in range(0, 3):
                if spatial_pher_neighborhood(map_pher, a, b, 1)[i, j] > maxi:
                    maxi == spatial_pher_neighborhood(map_pher, a, b, 1)[i, j]
                    maxi_line == i
                    maxi_column == j
    return [maxi_line, maxi_column]
        

def spatial_relocation (map, map_pher_home, map_pher_food,clock_map, a, b): #map : List[List[int]], a : int(ligne), b : int(colonne)
    """Préconditions : 0 <= a <= n-1  and 0 <= b <= m-1  and 0 <= c <= n-1  and 0 <= d <= m-1 and n >= 2*neigh and m >= 2*neigh and map[c,d] == 0
    Renvoie le monde dans lequel l'individu de coordonnées [a, b] a été déplacé aléatoirement si pas de phéromone, suit les phéromone_food
    si la fourmi est en quête, suit les phéromones_home si la fourmi a de la nourriture. Les matrices sont également modifiées en conséquent."""
    ant = map[a, b] #on enregistre la cellule à déplacer (une fourmi)
    pher_food_map = map_pher_food
    pher_home_map = map_pher_home
    relocation = map
    clock = clock_map
    move_line = a
    move_column = b
    if ant == "f": #déplacement des fourmis en recherche de nourriture
        pher_home_map = np.delete(pher_home_map, m*move_line + move_column)
        pher_home_map = np.insert(pher_home_map, m*move_line + move_column, pers_pher) # sur la map de phéromones "maison" la case de même coordonnées que la fourmi prend la valeur de la persistance des phéromones
        pher_home_map = np.array([[pher_home_map[m*j + i] for i in range(0, m)] for j in range(0, n)])
        if no_pheromone(map_pher_food, move_line, move_column): #il n'y a pas de phéromone "nourriture" à coté de la fourmi
            next_move = random.randint(0, 3) #on décide le prochain mouvement au hasard
            print(next_move)
            if next_move == 0:
                if move_line > 0 and relocation[move_line - 1, move_column] == " ": #la prochaine case existe et est vide
                # on supprime l'indidu du monde
                    relocation = np.delete(relocation, m*move_line + move_column) #on copie le monde sans l'individu
                    relocation = np.insert(relocation, m*move_line + move_column, " ") #on remplace la place de l'individu par une case vide (avec un " ")
                    relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
                # on remplace la case vide par l'individu
                    relocation = np.delete(relocation, m*(move_line - 1) + move_column) #on copie le monde sans la case vide
                    relocation = np.insert(relocation, m*(move_line - 1) + move_column, ant) #on remplace la place de la case vide par l'individu (ant)
                    relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
                    move_line = move_line - 1
                    move_column = move_column
            if next_move == 1:
                if move_column < m - 1 and relocation[move_line, move_column + 1] == " ":
                    relocation = np.delete(relocation, m*move_line + move_column) #on copie le monde sans l'individu
                    relocation = np.insert(relocation, m*move_line + move_column, " ") #on remplace la place de l'individu par une case vide (avec un " ")
                    relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
                    # on remplace la case vide par l'individu
                    relocation = np.delete(relocation, m*move_line + move_column + 1) #on copie le monde sans la case vide
                    relocation = np.insert(relocation, m*move_line + move_column + 1, ant) #on remplace la place de la case vide par l'individu (ant)
                    relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n) ]) 
                    move_line = move_line
                    move_column = move_column + 1
            if next_move == 2:
                if move_line < n - 1 and relocation[move_line + 1, move_column] == " ":
                    relocation = np.delete(relocation, m*move_line + move_column) #on copie le monde sans l'individu
                    relocation = np.insert(relocation, m*move_line + move_column, " ") #on remplace la place de l'individu par une case vide (avec un " ")
                    relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
                    # on remplace la case vide par l'individu
                    relocation = np.delete(relocation, m*(move_line + 1) + move_column) #on copie le monde sans la case vide
                    relocation = np.insert(relocation, m*(move_line + 1) + move_column, ant) #on remplace la place de la case vide par l'individu (ant)
                    relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n)]) 
                    move_line = move_line + 1
                    move_column = move_column
            if next_move == 3:
                if move_column > 0 and relocation[move_line, move_column - 1] == " ":
                    relocation = np.delete(relocation, m*move_line + move_column) #on copie le monde sans l'individu
                    relocation = np.insert(relocation, m*move_line + move_column, " ") #on remplace la place de l'individu par une case vide (avec un " ")
                    relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
                    # on remplace la case vide par l'individu
                    relocation = np.delete(relocation, m*move_line + move_column - 1) #on copie le monde sans la case vide
                    relocation = np.insert(relocation, m*move_line + move_column - 1, ant) #on remplace la place de la case vide par l'individu (ant)
                    relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n) ])
                    move_line = move_line
                    move_column = move_column - 1
 
        else : #l'individu va vers la case de son voisinage ayant la plus grande valeur de phéromone
            relocation = np.delete(relocation, m*move_line + move_column) #on copie le monde sans l'individu
            relocation = np.insert(relocation, m*move_line + move_column, " ") #on remplace la place de l'individu par une case vide (avec un " ")
            relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n)]) #on remet le monde sous forme de tableau
            if 0 < move_line < n-1 and 0 < move_column < m-1 and relocation[max_pheromone(map_pher_food, move_line, move_column)[0],  max_pheromone(map_pher_food, move_line, move_column)[1]] == " ": #pas de problème avec les bords
                relocation = np.delete(relocation, m*(move_line - 1 + max_pheromone(map_pher_food, move_line, move_column)[0]) + (move_column - 1 + max_pheromone(map_pher_food, move_line, move_column)[1]))
                relocation = np.insert(relocation, m*(move_line - 1 + max_pheromone(map_pher_food, a, move_column)[0]) + (move_column - 1 + max_pheromone(map_pher_food, a, move_column)[1]), "f")
                relocation = np.array([[relocation[m*k + l] for l in range(0, m)] for k in range(0, n)])
                move_line = move_line - 1 + max_pheromone(map_pher_food, a, move_column)[0]
                move_column = move_column - 1 + max_pheromone(map_pher_food, a, move_column)[1]
            if move_line == 0 and 0 < move_column < m-1 and relocation[max_pheromone(map_pher_food, move_line, move_column)[0],  max_pheromone(map_pher_food, move_line, move_column)[1]] == " ": # on a atteint le bord haut
                relocation = np.delete(relocation, m*(move_line + max_pheromone(map_pher_food, move_line, move_column)[0]) + (move_column - 1 + max_pheromone(map_pher_food, move_line, move_column)[1]))
                relocation = np.insert(relocation, m*(move_line + max_pheromone(map_pher_food, move_line, move_column)[0]) + (move_column - 1 + max_pheromone(map_pher_food, move_line, move_column)[1]), "f") 
                relocation = np.array([[relocation[m*k + l] for l in range(0, m)] for k in range(0, n)])
                move_line = move_line + max_pheromone(map_pher_food, a, move_column)[0]
                move_column = move_column - 1 + max_pheromone(map_pher_food, a, move_column)[1]
            if move_line == 0 and move_column == 0 and relocation[max_pheromone(map_pher_food, move_line, move_column)[0],  max_pheromone(map_pher_food, move_line, move_column)[1]] == " ": # on a atteint le bord haut gauche
                relocation = np.delete(relocation, m*(move_line + max_pheromone(map_pher_food, move_line, move_column)[0]) + (move_column + max_pheromone(map_pher_food, move_line, move_column)[1])) 
                relocation = np.insert(relocation, m*(move_line + max_pheromone(map_pher_food, move_line, move_column)[0]) + (move_column + max_pheromone(map_pher_food, move_line, move_column)[1]), "f")
                relocation = np.array([[relocation[m*k + l] for l in range(0, m)] for k in range(0, n)])
                move_line = move_line + max_pheromone(map_pher_food, a, move_column)[0]
                move_column = move_column + max_pheromone(map_pher_food, a, move_column)[1]
            if move_line == 0 and move_column == m-1 and relocation[max_pheromone(map_pher_food, move_line, move_column)[0],  max_pheromone(map_pher_food, move_line, move_column)[1]] == " ": #on a atteint le bord haut droit
                relocation = np.delete(relocation, m*(move_line + max_pheromone(map_pher_food, move_line, move_column)[0]) + (move_column - 1 + max_pheromone(map_pher_food, move_line, move_column)[1]))
                relocation = np.insert(relocation, m*(move_line + max_pheromone(map_pher_food, move_line, move_column)[0]) + (move_column - 1 + max_pheromone(map_pher_food, move_line, move_column)[1]), "f") 
                relocation = np.array([[relocation[m*k + l] for l in range(0, m)] for k in range(0, n)])
                move_line = move_line + max_pheromone(map_pher_food, a, move_column)[0]
                move_column = move_column - 1 + max_pheromone(map_pher_food, a, move_column)[1]
            if move_line == n-1 and 0 < move_column < m-1 and relocation[max_pheromone(map_pher_food, a, move_column)[0],  max_pheromone(map_pher_food, a, move_column)[1]] == " ": # on a atteint le bord bas
                relocation = np.delete(relocation, m*(move_line - 1 + max_pheromone(map_pher_food, move_line, move_column)[0]) + (move_column - 1 + max_pheromone(map_pher_food, move_line, move_column)[1]))
                relocation = np.insert(relocation, m*(move_line - 1 + max_pheromone(map_pher_food, move_line, move_column)[0]) + (move_column - 1 + max_pheromone(map_pher_food, move_line, move_column)[1]), "f")
                relocation = np.array([[relocation[m*k + l] for l in range(0, m)] for k in range(0, n)])
                move_line = move_line - 1 + max_pheromone(map_pher_food, a, move_column)[0]
                move_column = move_column - 1 + max_pheromone(map_pher_food, a, move_column)[1]
            if move_line == n-1 and move_column == 0 and relocation[max_pheromone(map_pher_food, move_line, move_column)[0],  max_pheromone(map_pher_food, move_line, move_column)[1]] == " ": # on a atteint le bord bas gauche
                relocation = np.delete(relocation, m*(move_line - 1 + max_pheromone(map_pher_food, move_line, move_column)[0]) + (move_column + max_pheromone(map_pher_food, move_line, move_column)[1]))
                relocation = np.insert(relocation, m*(move_line - 1 + max_pheromone(map_pher_food, move_line, move_column)[0]) + (move_column + max_pheromone(map_pher_food, move_line, move_column)[1]), "f") 
                relocation = np.array([[relocation[m*k + l] for l in range(0, m)] for k in range(0, n)])  
                move_line = move_line - 1 + max_pheromone(map_pher_food, a, move_column)[0]
                move_column = move_column + max_pheromone(map_pher_food, a, move_column)[1]
            if move_line == n-1 and move_column == m-1 and relocation[max_pheromone(map_pher_food, move_line, move_column)[0],  max_pheromone(map_pher_food, move_line, move_column)[1]] == " ": #on a atteint le bord bas droit
                relocation = np.delete(relocation, m*(move_line - 1 + max_pheromone(map_pher_food, move_line, move_column)[0]) + (move_column - 1 + max_pheromone(map_pher_food, move_line, move_column)[1]))
                relocation = np.insert(relocation, m*(move_line - 1 + max_pheromone(map_pher_food, move_line, move_column)[0]) + (move_column - 1 + max_pheromone(map_pher_food, move_line, move_column)[1]), "f") 
                relocation = np.array([[relocation[m*k + l] for l in range(0, m)] for k in range(0, n)])
                move_line = move_line - 1 + max_pheromone(map_pher_food, a, move_column)[0]
                move_column = move_column - 1 + max_pheromone(map_pher_food, a, move_column)[1]
                
        #on déplace et modifie l'horloge de la fourmi
        clock = np.delete(clock, m*move_line + move_column)
        clock = np.insert(clock, m*move_line + move_column, int(clock[m*a + b] - 1))
        if move_line != a and move_column != b :
            clock = np.delete(clock, m*a + b)
            clock = np.insert(clock, m*a + b, 0)
        clock = np.array([[clock[m*j + i] for i in range(0, m)] for j in range(0, n)])
        
    if ant == "s": #déplacement des fourmis ayant trouvé de la nourriture
        pher_food_map = np.delete(pher_home_map, m*move_line + move_column)
        pher_food_map = np.insert(pher_home_map, m*move_line + move_column, pers_pher) # sur la map de phéromones "nourriture" la case de même coordonnées que la fourmi prend la valeur de la persistance des phéromones
        pher_food_map = np.array([[pher_home_map[m*j + i] for i in range(0, m)] for j in range(0, n)])
        if no_pheromone(map_pher_home, move_line, move_column): #il n'y a pas de phéromone "maison" à coté de la fourmi
            next_move = random.randint(0, 3) #on décide le prochain mouvement au hasard
            if next_move == 0:
                if move_line > 0 and relocation[move_line - 1, move_column] == " ": #la prochaine case existe et est vide
                # on supprime l'indidu du monde
                    relocation = np.delete(relocation, m*move_line + move_column) #on copie le monde sans l'individu
                    relocation = np.insert(relocation, m*move_line + move_column, " ") #on remplace la place de l'individu par une case vide (avec un " ")
                    relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
                # on remplace la case vide par l'individu
                    relocation = np.delete(relocation, m*(move_line - 1) + move_column) #on copie le monde sans la case vide
                    relocation = np.insert(relocation, m*(move_line - 1) + move_column, ant) #on remplace la place de la case vide par l'individu (ant)
                    relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
                    move_line = move_line - 1
                    move_column = move_column
            if next_move == 1:
                if move_column < m - 1 and relocation[move_line, move_column + 1] == " ":
                    relocation = np.delete(relocation, m*move_line + move_column) #on copie le monde sans l'individu
                    relocation = np.insert(relocation, m*move_line + move_column, " ") #on remplace la place de l'individu par une case vide (avec un " ")
                    relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
                    # on remplace la case vide par l'individu
                    relocation = np.delete(relocation, m*move_line + move_column + 1) #on copie le monde sans la case vide
                    relocation = np.insert(relocation, m*move_line + move_column + 1, ant) #on remplace la place de la case vide par l'individu (ant)
                    relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n) ]) 
                    move_line = move_line
                    move_column = move_column + 1
            if next_move == 2:
                if move_line < n - 1 and relocation[move_line + 1, move_column] == " ":
                    relocation = np.delete(relocation, m*move_line + move_column) #on copie le monde sans l'individu
                    relocation = np.insert(relocation, m*move_line + move_column, " ") #on remplace la place de l'individu par une case vide (avec un " ")
                    relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
                    # on remplace la case vide par l'individu
                    relocation = np.delete(relocation, m*(move_line + 1) + move_column) #on copie le monde sans la case vide
                    relocation = np.insert(relocation, m*(move_line + 1) + move_column, ant) #on remplace la place de la case vide par l'individu (ant)
                    relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n)]) 
                    move_line = move_line + 1
                    move_column = move_column
            if next_move == 3:
                if move_column > 0 and relocation[move_line, move_column - 1] == " ":
                    relocation = np.delete(relocation, m*move_line + move_column) #on copie le monde sans l'individu
                    relocation = np.insert(relocation, m*move_line + move_column, " ") #on remplace la place de l'individu par une case vide (avec un " ")
                    relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n) ]) #on remet le monde sous forme de tableau
                    # on remplace la case vide par l'individu
                    relocation = np.delete(relocation, m*move_line + move_column - 1) #on copie le monde sans la case vide
                    relocation = np.insert(relocation, m*move_line + move_column - 1, ant) #on remplace la place de la case vide par l'individu (ant)
                    relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n) ])
                    move_line = move_line
                    move_column = move_column - 1   
                    
        else : #l'individu va vers la case de son voisinage ayant la plus grande valeur de phéromone
            relocation = np.delete(relocation, m*move_line + move_column) #on copie le monde sans l'individu
            relocation = np.insert(relocation, m*move_line + move_column, " ") #on remplace la place de l'individu par une case vide (avec un " ")
            relocation = np.array([[relocation[m*j + i] for i in range(0, m)] for j in range(0, n)]) #on remet le monde sous forme de tableau
            if 0 < move_line < n-1 and 0 < move_column < m-1 and relocation[max_pheromone(map_pher_home, move_line, move_column)[0],  max_pheromone(map_pher_home, move_line, move_column)[1]] == " ": #pas de problème avec les bords
                relocation = np.delete(relocation, m*(move_line - 1 + max_pheromone(map_pher_home, move_line, move_column)[0]) + (move_column - 1 + max_pheromone(map_pher_home, move_line, move_column)[1]))
                relocation = np.insert(relocation, m*(move_line - 1 + max_pheromone(map_pher_home, a, move_column)[0]) + (move_column - 1 + max_pheromone(map_pher_home, a, move_column)[1]), "f")
                relocation = np.array([[relocation[m*k + l] for l in range(0, m)] for k in range(0, n)])
                move_line = move_line - 1 + max_pheromone(map_pher_home, move_line, move_column)[0]
                move_column = move_column - 1 + max_pheromone(map_pher_home, a, move_column)[1]
            if move_line == 0 and 0 < move_column < m-1 and relocation[max_pheromone(map_pher_home, move_line, move_column)[0],  max_pheromone(map_pher_home, move_line, move_column)[1]] == " ": # on a atteint le bord haut
                relocation = np.delete(relocation, m*(move_line + max_pheromone(map_pher_home, move_line, move_column)[0]) + (move_column - 1 + max_pheromone(map_pher_home, move_line, move_column)[1]))
                relocation = np.insert(relocation, m*(move_line + max_pheromone(map_pher_home, move_line, move_column)[0]) + (move_column - 1 + max_pheromone(map_pher_home, move_line, move_column)[1]), "f") 
                relocation = np.array([[relocation[m*k + l] for l in range(0, m)] for k in range(0, n)])
                move_line = move_line + max_pheromone(map_pher_home, a, move_column)[0]
                move_column = move_column - 1 + max_pheromone(map_pher_home, move_line, move_column)[1]
            if move_line == 0 and move_column == 0 and relocation[max_pheromone(map_pher_home, move_line, move_column)[0],  max_pheromone(map_pher_home, move_line, move_column)[1]] == " ": # on a atteint le bord haut gauche
                relocation = np.delete(relocation, m*(move_line + max_pheromone(map_pher_home, move_line, move_column)[0]) + (move_column + max_pheromone(map_pher_home, move_line, move_column)[1])) 
                relocation = np.insert(relocation, m*(move_line + max_pheromone(map_pher_home, move_line, move_column)[0]) + (move_column + max_pheromone(map_pher_home, move_line, move_column)[1]), "f")
                relocation = np.array([[relocation[m*k + l] for l in range(0, m)] for k in range(0, n)])
                move_line = move_line + max_pheromone(map_pher_home, move_line, move_column)[0]
                move_column = move_column + max_pheromone(map_pher_home, move_line, move_column)[1]
            if move_line == 0 and move_column == m-1 and relocation[max_pheromone(map_pher_home, move_line, move_column)[0],  max_pheromone(map_pher_home, move_line, move_column)[1]] == " ": #on a atteint le bord haut droit
                relocation = np.delete(relocation, m*(move_line + max_pheromone(map_pher_home, move_line, move_column)[0]) + (move_column - 1 + max_pheromone(map_pher_home, move_line, move_column)[1]))
                relocation = np.insert(relocation, m*(move_line + max_pheromone(map_pher_home, move_line, move_column)[0]) + (move_column - 1 + max_pheromone(map_pher_home, move_line, move_column)[1]), "f") 
                relocation = np.array([[relocation[m*k + l] for l in range(0, m)] for k in range(0, n)])
                move_line = move_line + max_pheromone(map_pher_home, move_line, move_column)[0]
                move_column = move_column - 1 + max_pheromone(map_pher_home, move_line, move_column)[1]
            if move_line == n-1 and 0 < move_column < m-1 and relocation[max_pheromone(map_pher_home, move_line, move_column)[0],  max_pheromone(map_pher_home, move_line, move_column)[1]] == " ": # on a atteint le bord bas
                relocation = np.delete(relocation, m*(move_line - 1 + max_pheromone(map_pher_home, move_line, move_column)[0]) + (move_column - 1 + max_pheromone(map_pher_home, move_line, move_column)[1]))
                relocation = np.insert(relocation, m*(move_line - 1 + max_pheromone(map_pher_home, move_line, move_column)[0]) + (move_column - 1 + max_pheromone(map_pher_home, move_line, move_column)[1]), "f")
                relocation = np.array([[relocation[m*k + l] for l in range(0, m)] for k in range(0, n)])
                move_line = move_line - 1 + max_pheromone(map_pher_home, move_line, move_column)[0]
                move_column = move_column - 1 + max_pheromone(map_pher_home, move_line, move_column)[1]
            if move_line == n-1 and move_column == 0 and relocation[max_pheromone(map_pher_home, move_line, move_column)[0],  max_pheromone(map_pher_home, move_line, move_column)[1]] == " ": # on a atteint le bord bas gauche
                relocation = np.delete(relocation, m*(move_line - 1 + max_pheromone(map_pher_home, move_line, move_column)[0]) + (move_column + max_pheromone(map_pher_home, move_line, move_column)[1]))
                relocation = np.insert(relocation, m*(move_line - 1 + max_pheromone(map_pher_home, move_line, move_column)[0]) + (move_column + max_pheromone(map_pher_home, move_line, move_column)[1]), "f") 
                relocation = np.array([[relocation[m*k + l] for l in range(0, m)] for k in range(0, n)])  
                move_line = move_line - 1 + max_pheromone(map_pher_home, move_line, move_column)[0]
                move_column = move_column + max_pheromone(map_pher_home, move_line, move_column)[1]
            if move_line == n-1 and move_column == m-1 and relocation[max_pheromone(map_pher_home, move_line, move_column)[0],  max_pheromone(map_pher_home, move_line, move_column)[1]] == " ": #on a atteint le bord bas droit
                relocation = np.delete(relocation, m*(move_line - 1 + max_pheromone(map_pher_home, move_line, move_column)[0]) + (move_column - 1 + max_pheromone(map_pher_home, move_line, move_column)[1]))
                relocation = np.insert(relocation, m*(move_line - 1 + max_pheromone(map_pher_home, move_line, move_column)[0]) + (move_column - 1 + max_pheromone(map_pher_home, move_line, move_column)[1]), "f") 
                relocation = np.array([[relocation[m*k + l] for l in range(0, m)] for k in range(0, n)])
                move_line = move_line - 1 + max_pheromone(map_pher_home, move_line, move_column)[0]
                move_column = move_column - 1 + max_pheromone(map_pher_home, move_line, move_column)[1]
                
        #on déplace et modifie l'horloge de la fourmi
        clock = np.delete(clock, m*move_line + move_column)
        clock = np.insert(clock, m*move_line + move_column, int(clock[m*a + b] + 10))
        if move_line != a and move_column != b :
            clock = np.delete(clock, m*a + b)
            clock = np.insert(clock, m*a + b, 0)
        clock = np.array([[clock[m*j + i] for i in range(0, m)] for j in range(0, n)]) 
    print("map_pher_food :")
    print(pher_food_map)
    print("map_pher_home :")
    print(pher_home_map)
    print("clock :")
    print(clock)
    return relocation
    
print(spatial_relocation(ants(map1, 10), map_pher_home1, map_pher_food1, clock_map(ants(map1, 10)), 7, 7))
    
def no_food(map, a, b):
    """Précondition : 0 >= a >= n, 0 >= b >= m
    Renvoie True si il n'y a pas de nourriture dans le voisinage de la case et False sinon."""
    for i in spatial_ants_neighborhood(map, a, b, 1):
        for j in i:
            if j != " " and j != "h" and j != "f" and j != "s": #il y a de la nourriture dans le voisinage de la case, on peut utiliser le codage des caractères
                return False
    return True

def take_food(map, a, b):
    """Précondition : 0 >= a >= n, 0 >= b >= m
    Renvoie la même carte avec une unité de moins sur la nourriture"""
    food_map = map
    if food_map[a, b] == "f" or food_map[a, b] == "s": # la cellule sélectionnée est une fourmi
        if 0 < a < n-1 and 0 < b < m-1 : #pas de problème avec les bords
            for i in range(0, 3):
                for j in range(0, 3):
                    if spatial_ants_neighborhood(food_map, a, b, 1)[i, j] != " " and spatial_ants_neighborhood(food_map, a, b, 1)[i, j] != "h" and spatial_ants_neighborhood(food_map, a, b, 1)[i, j] != "f" and spatial_neighborhood(food_map, a, b, 1)[i, j] != "s": #la case [i, j] a de la nourriture
                        food_map = np.delete(food_map, m*(a - 1 + i) + (b - 1 + j)) #on copie le monde sans la case nourriture
                        food_map = np.insert(food_map, m*(a - 1 + i) + (b - 1 + j), str(int(spatial_neighborhood(map, a, b, 1)[i, j]) - 1)) #on enlève 1 à la valeur de nourriture
                        food_map = np.array([[food_map[m*k + l] for l in range(0, m)] for k in range(0, n) ])
                        return food_map
        if a == 0 and 0 < b < m-1: # on a atteint le bord haut
            for i in range(0, 2):
                for j in range(0, 3):
                    if spatial_ants_neighborhood(map, a, b, 1)[i, j] != " " and spatial_ants_neighborhood(map, a, b, 1)[i, j] != "h" and spatial_ants_neighborhood(map, a, b, 1)[i, j] != "f" and spatial_neighborhood(map, a, b, 1)[i, j] != "s": #la case [i, j] a de la nourriture
                        food_map = np.delete(food_map, m*(a + i) + (b - 1 + j)) #on copie le monde sans la case nourriture
                        food_map = np.insert(food_map, m*(a + i) + (b - 1 + j), str(int(spatial_ants_neighborhood(map, a, b, 1)[i, j]) - 1)) #on enlève 1 à la valeur de nourriture
                        food_map = np.array([[food_map[m*k + l] for l in range(0, m)] for k in range(0, n)])
                        return food_map
        if a == 0 and b == 0: # on a atteint le bord haut gauche
            for i in range(0, 2):
                for j in range(0, 2):
                    if spatial_ants_neighborhood(map, a, b, 1)[i, j] != " " and spatial_ants_neighborhood(map, a, b, 1)[i, j] != "h" and spatial_ants_neighborhood(map, a, b, 1)[i, j] != "f" and spatial_neighborhood(map, a, b, 1)[i, j] != "s": #la case [i, j] a de la nourriture
                        food_map = np.delete(food_map, m*(a + i) + (b + j)) #on copie le monde sans la case nourriture
                        food_map = np.insert(food_map, m*(a + i) + (b + j), str(int(spatial_neighborhood(map, a, b, 1)[i, j]) - 1)) #on enlève 1 à la valeur de nourriture
                        food_map = np.array([[food_map[m*k + l] for l in range(0, m)] for k in range(0, n)])
                        return food_map
        if a == 0 and b == m-1: #on a atteint le bord haut droit
            for i in range(0, 2):
                for j in range(0, 2):
                    if spatial_ants_neighborhood(map, a, b, 1)[i, j] != " " and spatial_ants_neighborhood(map, a, b, 1)[i, j] != "h" and spatial_ants_neighborhood(map, a, b, 1)[i, j] != "f" and spatial_neighborhood(map, a, b, 1)[i, j] != "s": #la case [i, j] a de la nourriture
                        food_map = np.delete(food_map, m*(a + i) + (b - 1 + j)) #on copie le monde sans la case nourriture
                        food_map = np.insert(food_map, m*(a + i) + (b - 1 + j), str(int(spatial_neighborhood(map, a, b, 1)[i, j]) - 1)) #on enlève 1 à la valeur de nourriture
                        food_map = np.array([[food_map[m*k + l] for l in range(0, m)] for k in range(0, n)])
                        return food_map
        if a == n-1 and 0 < b < m-1: # on a atteint le bord bas
            for i in range(0, 2):
                for j in range(0, 3):
                    if spatial_ants_neighborhood(map, a, b, 1)[i, j] != " " and spatial_ants_neighborhood(map, a, b, 1)[i, j] != "h" and spatial_ants_neighborhood(map, a, b, 1)[i, j] != "f" and spatial_neighborhood(map, a, b, 1)[i, j] != "s": #la case [i, j] a de la nourriture
                        food_map = np.delete(food_map, m*(a - 1 + i) + (b - 1 + j)) #on copie le monde sans la case nourriture
                        food_map = np.insert(food_map, m*(a - 1 + i) + (b - 1 + j), str(int(spatial_neighborhood(map, a, b, 1)[i, j]) - 1)) #on enlève 1 à la valeur de nourriture
                        food_map = np.array([[food_map[m*k + l] for l in range(0, m)] for k in range(0, n)])
                        return food_map
        if a == n-1 and b == 0: # on a atteint le bord bas gauche
            for i in range(0, 2):
                for j in range(0, 2):
                    if spatial_ants_neighborhood(map, a, b, 1)[i, j] != " " and spatial_ants_neighborhood(map, a, b, 1)[i, j] != "h" and spatial_ants_neighborhood(map, a, b, 1)[i, j] != "f" and spatial_neighborhood(map, a, b, 1)[i, j] != "s": #la case [i, j] a de la nourriture
                        food_map = np.delete(food_map, m*(a - 1 + i) + (b + j)) #on copie le monde sans la case nourriture
                        food_map = np.insert(food_map, m*(a - 1 + i) + (b + j), str(int(spatial_ants_neighborhood(map, a, b, 1)[i, j]) - 1)) #on enlève 1 à la valeur de nourriture
                        food_map = np.array([[food_map[m*k + l] for l in range(0, m)] for k in range(0, n)])
                        return food_map
        if a == n-1 and b == m-1: #on a atteint le bord bas droit
            for i in range(0, 2):
                for j in range(0, 2):
                    if spatial_ants_neighborhood(map, a, b, 1)[i, j] != " " and spatial_ants_neighborhood(map, a, b, 1)[i, j] != "h" and spatial_ants_neighborhood(map, a, b, 1)[i, j] != "f" and spatial_neighborhood(map, a, b, 1)[i, j] != "s": #la case [i, j] a de la nourriture
                        food_map = np.delete(food_map, m*(a - 1 + i) + (b - 1 + j)) #on copie le monde sans la case nourriture
                        food_map = np.insert(food_map, m*(a - 1 + i) + (b - 1 + j), str(int(spatial_ants_neighborhood(map, a, b, 1)[i, j]) - 1)) #on enlève 1 à la valeur de nourriture
                        food_map = np.array([[food_map[m*k + l] for l in range(0, m)] for k in range(0, n)])
                        return food_map
    return food_map

#print(take_food(np.array([[3, "f", "s", 4, "h", "h", "h", "h", "h", "h", "s", "h", "h", "h", "h", "h", "h", "h"] for i in range(0, 18)]), 0, 2))

def no_home(map, a, b):
    """Précondition : 0 >= a >= n, 0 >= b >= m
    Renvoie True si il n'y a pas de case fourmilière dans le voisinage et False sinon."""
    if map[a, b] == "s" or map[a, b] == "f":
        for i in spatial_ants_neighborhood(map, a, b, 1):
            for j in i:
                if j == "h": #il n'y a pas de case fourmilière à côté
                    return False
    return True

#print(no_home(map, 1, 9))

def one_food_quest (map, a, b):
    """Précondition : 0 >= a >= n, 0 >= b >= m
    Si la case [a, b] est une fourmi, renvoie la matrice après son déplacement aléatoire sans retourner sur ses pas
    sinon renvoie la matrice initiale"""
    compt = 20
    move_map = map
    move_line = a #ligne de la fourmi
    move_column = b #colonne de la fourmi
    previous_location = - 10 #on retient la position précédente pour ne pas y revenir
    if move_map[a, b] == "f" : #la case sélectionnée est une fourmi
        while no_food(move_map, move_line, move_column) and compt > 0: #il n'y a pas de nourriture à coté de la fourmi            
            next_move = random.randint(0, 3) #on décide le procahin mouvement au hasard
            if next_move == 0 and previous_location != 2:
                if move_line > 0 and move_map[move_line - 1, move_column] == " ": #la prochaine case existe et est vide
                    move_map = spatial_relocation(move_map, move_line, move_column, move_line - 1, move_column) #déplacement vers le haut
                    previous_location = next_move
                    move_line = move_line - 1
            if next_move == 1 and previous_location != 3:
                if move_column < m - 1 and move_map[move_line, move_column + 1] == " ":
                    move_map = spatial_relocation(move_map, move_line, move_column, move_line, move_column + 1) #déplacement vers la droite
                    previous_location = next_move                  
                    move_column = move_column + 1
            if next_move == 2 and previous_location != 0:
                if move_line < n - 1 and move_map[move_line + 1, move_column] == " ":
                    move_map = spatial_relocation(move_map, move_line, move_column, move_line + 1, move_column) #déplacement vers le bas
                    previous_location = next_move
                    move_line = move_line + 1
            if next_move == 3 and previous_location != 1:
                if move_column > 0 and move_map[move_line, move_column - 1] == " ":
                    move_map = spatial_relocation(move_map, move_line, move_column, move_line, move_column - 1) #déplacement vers la gauche
                    previous_location = next_move
                    move_column = move_column - 1
            print(no_food(move_map, move_line, move_column), move_line, move_column)
            compt = compt - 1
            print(move_map)
    return move_map
            

def ants_food_quest(map):
    
    """Effectue le déplacement aléatoire de toutes les fourmis jusqu'à ce qu'il y en ait une qui trouve de la nourriture"""
    move_map = map
    move_line = - 1
    move_column = - 1
    food = False
    while food == False:
        for i in range(0, n):
            for j in range(0, m):
                if move_map[i, j] == "f" : #la case sélectionnée est une fourmi
                    next_move = random.randint(0, 3) #on décide le prochain mouvement au hasard
                    if next_move == 0:
                        if i > 0 and move_map[i - 1, j] == " ": #la prochaine case existe et est vide
                            move_map = spatial_relocation(move_map, i, j, i - 1, j) #déplacement vers le haut
                            move_line = i - 1
                            move_column = j
                    if next_move == 1:
                        if j < m - 1 and move_map[i, j + 1] == " ":
                            move_map = spatial_relocation(move_map, i, j, i, j + 1) #déplacement vers la droite
                            move_line = i
                            move_column = j + 1
                    if next_move == 2:
                        if i < n - 1 and move_map[i + 1, j] == " ":
                            move_map = spatial_relocation(move_map, i, j, i + 1, j) #déplacement vers le bas
                            move_line = i + 1
                            move_column = j
                    if next_move == 3:
                        if j > 0 and move_map[i, j - 1] == " ":
                            move_map = spatial_relocation(move_map, i, j, i, j - 1) #déplacement vers la gauche
                            move_line = i
                            move_column = j - 1
                    if no_food(move_map, move_line, move_column) == False:
                        food = True
        print(move_map)
        print(food)
    return move_map

#print(one_food_quest(ants(generate_simple_map(n, m), 1), 3, 3))
#print(ants_food_quest(ants(generate_simple_map(m, n), 10)))

def return_home (map):
    """Si une fourmi a trouvé de la nourriture, effectue le déplacement de celle-ci vers la fourmilière"""
    move_map = map
    line_found = - 1 #ligne de la fourmi découvreuse de nourriture
    column_found = - 1 #colonne de la fourmi découvreuse de nourriture
    eratic_move = 0
    next_move = 0
    for i in range(0, n):
        for j in range(0, m):
            if move_map[i, j] == "f" :
                if no_food(move_map, i, j) == False:
                    line_found = i
                    column_found = j
                    move_map = np.delete(move_map, m*line_found + column_found) #on copie le monde sans l'individu
                    move_map = np.insert(move_map, m*line_found + column_found, "s") #on remplace la fourmi par une fourmi ayant de la nourriture (avec un "s")
                    move_map = np.array([[move_map[m*k + l] for l in range(0, m)] for k in range(0, n) ]) #on remet le monde sous forme de tableau
                    move_map = take_food(move_map, line_found, column_found)
                    while no_home(move_map, line_found, column_found):
                        # mouvement diagonal vers la fourmilière
                        if line_found >= n/2 and column_found >= m/2 and move_map[line_found - 1, column_found - 1] == " ": #la fourmi est dans le quart bas droit de la map
                            move_map = spatial_relocation(move_map, line_found, column_found, line_found - 1, column_found - 1)
                            line_found = line_found - 1
                            column_found = column_found - 1
                        if line_found >= n/2 and column_found <= m/2 and move_map[line_found - 1, column_found + 1] == " ": #la fourmi est dans le quart bas gauche de la map
                            move_map = spatial_relocation(move_map, line_found, column_found, line_found - 1, column_found + 1)
                            line_found = line_found - 1
                            column_found = column_found + 1  
                        if line_found <= n/2 and column_found <= m/2 and move_map[line_found + 1, column_found + 1] == " ": #la fourmi est dans le quart haut gauche de la map
                            move_map = spatial_relocation(move_map, line_found, column_found, line_found + 1, column_found + 1)
                            line_found = line_found + 1
                            column_found = column_found + 1   
                        if line_found <= n/2 and column_found >= m/2 and move_map[line_found + 1, column_found - 1] == " ": #la fourmi est dans le quart haut droit de la map
                            move_map = spatial_relocation(move_map, line_found, column_found, line_found + 1, column_found - 1)
                            line_found = line_found + 1
                            column_found = column_found - 1   
                        eratic_move = eratic_move + 1
                        print(move_map)
                    
                        # mouvement eratique périodique
                        if eratic_move > 2:
                            next_move = random.randint(0, 3) #on décide le procahin mouvement au hasard
                            if next_move == 0:
                                if line_found > 0 and move_map[line_found - 1, column_found] == " ": #la prochaine case existe et est vide
                                    move_map = spatial_relocation(move_map, line_found, column_found, line_found - 1, column_found) #déplacement vers le haut
                                    line_found = line_found - 1
                            if next_move == 1:
                                if column_found < m - 1 and move_map[line_found, column_found + 1] == " ":
                                    move_map = spatial_relocation(move_map, line_found, column_found, line_found, column_found + 1) #déplacement vers la droite
                                    column_found = column_found + 1
                            if next_move == 2:
                                if line_found < n - 1 and move_map[line_found + 1, column_found] == " ":
                                    move_map = spatial_relocation(move_map, line_found, column_found, line_found + 1, column_found) #déplacement vers le bas
                                    line_found = line_found + 1
                            if next_move == 3:
                                if column_found > 0 and move_map[line_found, column_found - 1] == " ":
                                    move_map = spatial_relocation(move_map, line_found, column_found, line_found, column_found - 1) #déplacement vers la gauche
                                    column_found = column_found - 1
                            print(move_map)
                            eratic_move = 0
                    print(1)
                    move_map = np.delete(move_map, m*line_found + column_found) #on copie le monde sans l'individu
                    move_map = np.insert(move_map, m*line_found + column_found, "f") #on remplace la fourmi par une fourmi sans nourriture (avec un "f")
                    move_map = np.array([[move_map[m*k + l] for l in range(0, m)] for k in range(0, n) ])
    return move_map

#print(return_home(ants_food_quest(ants(map, 10))))



3
[[' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'h' 'h' ' ' ' ' ' ' ' ' ' ' ' ' ' ' '3']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' 'h' 'h' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ' ']
 [' ' ' ' ' ' ' ' ' ' '

In [ ]:
def one_turn(map):
    """Effectue un tour de la simulation complète"""
    move_map = map
    move_line = - 1
    move_column = - 1
    for i in range(0, n):
        for j in range(0, m):
             if move_map[i, j] == "f" : #la case sélectionnée est une fourmi
                next_move = random.randint(0, 3) #on décide le prochain mouvement au hasard
                if next_move == 0:
                    if i > 0 and move_map[i - 1, j] == " ": #la prochaine case existe et est vide
                        move_map = spatial_relocation(move_map, i, j, i - 1, j) #déplacement vers le haut
                        move_line = i - 1
                        move_column = j
                if next_move == 1:
                    if j < m - 1 and move_map[i, j + 1] == " ":
                        move_map = spatial_relocation(move_map, i, j, i, j + 1) #déplacement vers la droite
                        move_line = i
                        move_column = j + 1
                if next_move == 2:
                    if i < n - 1 and move_map[i + 1, j] == " ":
                        move_map = spatial_relocation(move_map, i, j, i + 1, j) #déplacement vers le bas
                        move_line = i + 1
                        move_column = j
                if next_move == 3:
                    if j > 0 and move_map[i, j - 1] == " ":
                        move_map = spatial_relocation(move_map, i, j, i, j - 1) #déplacement vers la gauche
                        move_line = i
                        move_column = j - 1
                if no_food(move_map, move_line, move_column) == False:
                    eratic_move = 0
                    next_move = 0
                    if move_map[move_line, move_column] == "f" :
                        move_map = np.delete(move_map, m*move_line + move_column) #on copie le monde sans l'individu
                        move_map = np.insert(move_map, m*move_line + move_column, "s") #on remplace la fourmi par une fourmi ayant de la nourriture (avec un "s")
                        move_map = np.array([[move_map[m*k + l] for l in range(0, m)] for k in range(0, n) ]) #on remet le monde sous forme de tableau
                        move_map = take_food(move_map, move_line, move_column)

                        # mouvement eratique périodique
                        if eratic_move > 2:
                            next_move = random.randint(0, 3) #on décide le procahin mouvement au hasard
                            if next_move == 0:
                                if line_found > 0 and move_map[line_found - 1, column_found] == " ": #la prochaine case existe et est vide
                                    move_map = spatial_relocation(move_map, line_found, column_found, line_found - 1, column_found) #déplacement vers le haut
                                    line_found = line_found - 1
                            if next_move == 1:
                                if column_found < m - 1 and move_map[line_found, column_found + 1] == " ":
                                    move_map = spatial_relocation(move_map, line_found, column_found, line_found, column_found + 1) #déplacement vers la droite
                                    column_found = column_found + 1
                            if next_move == 2:
                                if line_found < n - 1 and move_map[line_found + 1, column_found] == " ":
                                    move_map = spatial_relocation(move_map, line_found, column_found, line_found + 1, column_found) #déplacement vers le bas
                                    line_found = line_found + 1
                            if next_move == 3:
                                if column_found > 0 and move_map[line_found, column_found - 1] == " ":
                                    move_map = spatial_relocation(move_map, line_found, column_found, line_found, column_found - 1) #déplacement vers la gauche
                                    column_found = column_found - 1
                            print(move_map)
                            eratic_move = 0
                    move_map = np.delete(move_map, m*line_found + column_found) #on copie le monde sans l'individu
                    move_map = np.insert(move_map, m*line_found + column_found, "f") #on remplace la fourmi par une fourmi sans nourriture (avec un "f")
                    move_map = np.array([[move_map[m*k + l] for l in range(0, m)] for k in range(0, n) ])
    return move_map

        print(move_map)
        print(food)
    return move_map
                    